In [1]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

import pandas as pd

In [3]:
import transformers
print(transformers.__version__)

4.40.0


In [4]:
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", device= "cpu")

In [6]:
data = pd.read_csv('bdd.csv')
data

,page_number,sentence_chunk,tokens_number
0,1,1 Préambule Après onze années en tant que ch...,67.75
1,1,Deux choses qu’on ne nous apprend pas à l’éco...,56.00
2,1,J’ai pratiquement fait le tour des livres et ...,17.00
3,2,"2 Pour finir, je pense avoir synthétisé ma mo...",10.75
4,13,Traduire un modèle entité/association vers un ...,41.00
...,...,...,...
504,148,Ceci via une interface intuitive et très facil...,2.75
505,148,La version actuelle offre un ensemble de fonct...,3.75
506,148,Frai est le premier outil dans son genre. . . ...,16.50
507,148,Les cinq agrégats de base sont également implé...,4.00


In [10]:
list_chunk = [item for item in data['sentence_chunk']]
len(list_chunk)

509

In [12]:
embedding_model.to("cpu")

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [14]:
%%time

#embed all texts in batchs of 32 
text_chunk_embeddings = embedding_model.encode(sentences=list_chunk,
                                               batch_size = 32,
                                               convert_to_tensor=True )
text_chunk_embeddings

CPU times: total: 2min 12s
Wall time: 1min 45s


tensor([[ 0.0084, -0.0511, -0.0165,  ...,  0.0472, -0.0016, -0.0583],
        [ 0.0154, -0.0636, -0.0172,  ...,  0.0636, -0.0237, -0.0458],
        [-0.0179,  0.0115, -0.0171,  ...,  0.0487,  0.0070, -0.0363],
        ...,
        [-0.0328,  0.0041, -0.0186,  ...,  0.0311,  0.0331, -0.0055],
        [-0.0670, -0.0206, -0.0211,  ...,  0.0174, -0.0118, -0.0345],
        [-0.0031, -0.0320, -0.0195,  ...,  0.0783,  0.0331, -0.0453]])

In [17]:
text_chunk_embeddings.shape

torch.Size([509, 768])

In [24]:
#save the embeddings 
embedding_values = text_chunk_embeddings.tolist()
data['embeddings'] = embedding_values
data.head()

data.to_csv("bdd.csv", index=False)